In [ ]:
def split_data_into_segments(data, segment_duration=20.0, dt=0.5):
    segments = []
    
    points_per_segment = int(segment_duration / dt)
    
    total_points = len(data['vt'])
    unique_vehicles = np.unique(data['id_idx'])
    
    print(f"Total data points: {total_points}, Points per segment: {points_per_segment}")
    print(f"Expected segments: {total_points // points_per_segment}")
    
    for start_idx in range(0, total_points - points_per_segment + 1, points_per_segment):
        end_idx = start_idx + points_per_segment
        
        segment_data = {
            'vt': data['vt'][start_idx:end_idx],
            's': data['s'][start_idx:end_idx],
            'dv': data['dv'][start_idx:end_idx],
            'label_v': data['label_v'][start_idx:end_idx],
            'id_idx': data['id_idx'][start_idx:end_idx],
            'n_vehicles': data['n_vehicles'],
            'tracks': {}
        }
        
        for veh_id in unique_vehicles:
            veh_mask = (segment_data['id_idx'] == veh_id)
            if np.sum(veh_mask) > 0:
                segment_data['tracks'][veh_id] = {
                    'vFollReal': segment_data['vt'][veh_mask],
                    'vLeadReal': np.zeros(np.sum(veh_mask)),
                    'sReal': segment_data['s'][veh_mask],
                    'dvReal': segment_data['dv'][veh_mask],
                    'vFollReal_next': np.zeros(np.sum(veh_mask)),
                    'last_vt': segment_data['vt'][veh_mask][0] if np.sum(veh_mask) > 0 else 0.0,
                    'last_s': segment_data['s'][veh_mask][0] if np.sum(veh_mask) > 0 else 10.0
                }
        
        segments.append(segment_data)
    
    print(f"Actual segments created: {len(segments)}")
    return segments

In [ ]:
def validate_ar_model_segments(trace, model, train_data, val_segments, n_samples=50):

    print(f"Starting validation on {len(val_segments)} segments...")
    
    all_segments_speed_metrics = []
    all_segments_spacing_metrics = []
    all_segments_acceleration_metrics = []
    
    for seg_idx, segment_data in enumerate(val_segments):
        print(f"\nProcessing segment {seg_idx + 1}/{len(val_segments)}...")
        
        try:
            segment_results = validate_ar_model_comprehensive_improved(
                trace, model, train_data, segment_data, n_samples=n_samples
            )
            
            all_segments_speed_metrics.append(segment_results['speed_metrics'])
            all_segments_spacing_metrics.append(segment_results['spacing_metrics'])
            all_segments_acceleration_metrics.append(segment_results['acceleration_metrics'])
            
            print(f"Segment {seg_idx + 1} completed - "
                  f"Speed RMSE: {segment_results['speed_metrics']['rmse']:.4f}, "
                  f"Spacing RMSE: {segment_results['spacing_metrics']['rmse']:.4f}, "
                  f"Acceleration RMSE: {segment_results['acceleration_metrics']['rmse']:.4f}")
                  
        except Exception as e:
            print(f"Segment {seg_idx + 1} validation failed: {e}")
            continue
    
    if all_segments_speed_metrics:
        avg_speed_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_speed_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_speed_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_speed_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_speed_metrics])
        }
        
        avg_spacing_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_spacing_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_spacing_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_spacing_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_spacing_metrics])
        }
        
        avg_acceleration_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_acceleration_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_acceleration_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_acceleration_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_acceleration_metrics])
        }
        
        std_speed_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_speed_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_speed_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_speed_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_speed_metrics])
        }
        
        std_spacing_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_spacing_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_spacing_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_spacing_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_spacing_metrics])
        }
        
        std_acceleration_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_acceleration_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_acceleration_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_acceleration_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_acceleration_metrics])
        }
    else:
        print("No successful segment validation results!")
        return None
    
    
    print(f"\nAverage Speed Metrics:")
    print(f"  RMSE: {avg_speed_metrics['rmse']:.4f} ± {std_speed_metrics['rmse']:.4f} m/s")
    print(f"  NRMSE: {avg_speed_metrics['nrmse']:.4f} ± {std_speed_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_speed_metrics['mae']:.4f} ± {std_speed_metrics['mae']:.4f} m/s")
    
    print(f"\nAverage Spacing Metrics:")
    print(f"  RMSE: {avg_spacing_metrics['rmse']:.4f} ± {std_spacing_metrics['rmse']:.4f} m")
    print(f"  NRMSE: {avg_spacing_metrics['nrmse']:.4f} ± {std_spacing_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_spacing_metrics['mae']:.4f} ± {std_spacing_metrics['mae']:.4f} m")
    
    print(f"\nAverage Acceleration Metrics:")
    print(f"  RMSE: {avg_acceleration_metrics['rmse']:.4f} ± {std_acceleration_metrics['rmse']:.4f} m/s²")
    print(f"  NRMSE: {avg_acceleration_metrics['nrmse']:.4f} ± {std_acceleration_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_acceleration_metrics['mae']:.4f} ± {std_acceleration_metrics['mae']:.4f} m/s²")
    
    return {
        'avg_speed_metrics': avg_speed_metrics,
        'avg_spacing_metrics': avg_spacing_metrics,
        'avg_acceleration_metrics': avg_acceleration_metrics,
        'std_speed_metrics': std_speed_metrics,
        'std_spacing_metrics': std_spacing_metrics,
        'std_acceleration_metrics': std_acceleration_metrics,
        'all_segments_speed_metrics': all_segments_speed_metrics,
        'all_segments_spacing_metrics': all_segments_spacing_metrics,
        'all_segments_acceleration_metrics': all_segments_acceleration_metrics,
        'successful_segments': len(all_segments_speed_metrics),
        'total_segments': len(val_segments)
    }

In [ ]:
def validate_dynamic_ar_model_segments(trace, model, train_data, val_segments, rho_block_info, n_samples=50, d=1, step=1, tau_sec=5):


    all_segments_speed_metrics = []
    all_segments_spacing_metrics = []
    all_segments_acceleration_metrics = []
    all_segments_vehicle_metrics = []
    
    for seg_idx, segment_data in enumerate(val_segments):
        print(f"\nProcessing segment {seg_idx + 1}/{len(val_segments)}...")
        
        try:
            segment_results = validate_dynamic_ar_model_comprehensive(
                trace, model, train_data, segment_data, rho_block_info, 
                n_samples=n_samples, d=d, step=step, tau_sec=tau_sec
            )
            
            all_segments_speed_metrics.append(segment_results['speed_metrics'])
            all_segments_spacing_metrics.append(segment_results['spacing_metrics'])
            all_segments_acceleration_metrics.append(segment_results['acceleration_metrics'])
            all_segments_vehicle_metrics.append(segment_results['vehicle_metrics'])
            
            print(f"Segment {seg_idx + 1} completed - "
                  f"Speed RMSE: {segment_results['speed_metrics']['rmse']:.4f}, "
                  f"Spacing RMSE: {segment_results['spacing_metrics']['rmse']:.4f}, "
                  f"Acceleration RMSE: {segment_results['acceleration_metrics']['rmse']:.4f}")
                  
        except Exception as e:
            print(f"Segment {seg_idx + 1} validation failed: {e}")
            continue
    
    if all_segments_speed_metrics:
        avg_speed_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_speed_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_speed_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_speed_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_speed_metrics])
        }
        
        avg_spacing_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_spacing_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_spacing_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_spacing_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_spacing_metrics])
        }
        
        avg_acceleration_metrics = {
            'mse': np.mean([m['mse'] for m in all_segments_acceleration_metrics]),
            'rmse': np.mean([m['rmse'] for m in all_segments_acceleration_metrics]),
            'mae': np.mean([m['mae'] for m in all_segments_acceleration_metrics]),
            'nrmse': np.mean([m['nrmse'] for m in all_segments_acceleration_metrics])
        }
        
        std_speed_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_speed_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_speed_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_speed_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_speed_metrics])
        }
        
        std_spacing_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_spacing_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_spacing_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_spacing_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_spacing_metrics])
        }
        
        std_acceleration_metrics = {
            'mse': np.std([m['mse'] for m in all_segments_acceleration_metrics]),
            'rmse': np.std([m['rmse'] for m in all_segments_acceleration_metrics]),
            'mae': np.std([m['mae'] for m in all_segments_acceleration_metrics]),
            'nrmse': np.std([m['nrmse'] for m in all_segments_acceleration_metrics])
        }
        
        all_vehicles_speed_rmse = []
        all_vehicles_spacing_rmse = []
        all_vehicles_accel_rmse = []
        
        for seg_vehicle_metrics in all_segments_vehicle_metrics:
            for veh_id, metrics in seg_vehicle_metrics.items():
                all_vehicles_speed_rmse.append(metrics['speed']['rmse'])
                all_vehicles_spacing_rmse.append(metrics['spacing']['rmse'])
                all_vehicles_accel_rmse.append(metrics['acceleration']['rmse'])
        
        avg_vehicle_speed_rmse = np.mean(all_vehicles_speed_rmse) if all_vehicles_speed_rmse else 0
        avg_vehicle_spacing_rmse = np.mean(all_vehicles_spacing_rmse) if all_vehicles_spacing_rmse else 0
        avg_vehicle_accel_rmse = np.mean(all_vehicles_accel_rmse) if all_vehicles_accel_rmse else 0
        
        std_vehicle_speed_rmse = np.std(all_vehicles_speed_rmse) if all_vehicles_speed_rmse else 0
        std_vehicle_spacing_rmse = np.std(all_vehicles_spacing_rmse) if all_vehicles_spacing_rmse else 0
        std_vehicle_accel_rmse = np.std(all_vehicles_accel_rmse) if all_vehicles_accel_rmse else 0
        
    else:
        print("No successful segment validation results!")
        return None
    
    print(f"Successful segments: {len(all_segments_speed_metrics)}/{len(val_segments)}")
    print(f"AR order: {d}, Block size: {tau_sec} seconds")
    
    print(f"\nAverage Speed Metrics:")
    print(f"  RMSE: {avg_speed_metrics['rmse']:.4f} ± {std_speed_metrics['rmse']:.4f} m/s")
    print(f"  NRMSE: {avg_speed_metrics['nrmse']:.4f} ± {std_speed_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_speed_metrics['mae']:.4f} ± {std_speed_metrics['mae']:.4f} m/s")
    
    print(f"\nAverage Spacing Metrics:")
    print(f"  RMSE: {avg_spacing_metrics['rmse']:.4f} ± {std_spacing_metrics['rmse']:.4f} m")
    print(f"  NRMSE: {avg_spacing_metrics['nrmse']:.4f} ± {std_spacing_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_spacing_metrics['mae']:.4f} ± {std_spacing_metrics['mae']:.4f} m")
    
    print(f"\nAverage Acceleration Metrics:")
    print(f"  RMSE: {avg_acceleration_metrics['rmse']:.4f} ± {std_acceleration_metrics['rmse']:.4f} m/s²")
    print(f"  NRMSE: {avg_acceleration_metrics['nrmse']:.4f} ± {std_acceleration_metrics['nrmse']:.4f}")
    print(f"  MAE: {avg_acceleration_metrics['mae']:.4f} ± {std_acceleration_metrics['mae']:.4f} m/s²")
    
    print(f"\nVehicle-level Average Metrics:")
    print(f"  Speed RMSE: {avg_vehicle_speed_rmse:.4f} ± {std_vehicle_speed_rmse:.4f} m/s")
    print(f"  Spacing RMSE: {avg_vehicle_spacing_rmse:.4f} ± {std_vehicle_spacing_rmse:.4f} m")
    print(f"  Acceleration RMSE: {avg_vehicle_accel_rmse:.4f} ± {std_vehicle_accel_rmse:.4f} m/s²")
    
    return {
        'avg_speed_metrics': avg_speed_metrics,
        'avg_spacing_metrics': avg_spacing_metrics,
        'avg_acceleration_metrics': avg_acceleration_metrics,
        'std_speed_metrics': std_speed_metrics,
        'std_spacing_metrics': std_spacing_metrics,
        'std_acceleration_metrics': std_acceleration_metrics,
        'avg_vehicle_speed_rmse': avg_vehicle_speed_rmse,
        'avg_vehicle_spacing_rmse': avg_vehicle_spacing_rmse,
        'avg_vehicle_accel_rmse': avg_vehicle_accel_rmse,
        'std_vehicle_speed_rmse': std_vehicle_speed_rmse,
        'std_vehicle_spacing_rmse': std_vehicle_spacing_rmse,
        'std_vehicle_accel_rmse': std_vehicle_accel_rmse,
        'all_segments_speed_metrics': all_segments_speed_metrics,
        'all_segments_spacing_metrics': all_segments_spacing_metrics,
        'all_segments_acceleration_metrics': all_segments_acceleration_metrics,
        'all_segments_vehicle_metrics': all_segments_vehicle_metrics,
        'successful_segments': len(all_segments_speed_metrics),
        'total_segments': len(val_segments),
        'ar_order': d,
        'tau_sec': tau_sec
    }

In [ ]:
def run_ar_segment_validation(file_path, trace, model, train_data, segment_duration=20.0, n_samples=50):

    
    test_data = prepare_ar_idm_data(file_path, dt_target=0.5)

    segments = split_data_into_segments(test_data, segment_duration=segment_duration, dt=0.5)
    

    results = validate_ar_model_segments(trace, model, train_data, segments, n_samples=n_samples)
    
    if results:
        print("\n" + "="*80)
        print("FINAL RESULTS")
        print("="*80)
        print(f"Overall average speed RMSE: {results['avg_speed_metrics']['rmse']:.4f} ± {results['std_speed_metrics']['rmse']:.4f} m/s")
        print(f"Overall average spacing RMSE: {results['avg_spacing_metrics']['rmse']:.4f} ± {results['std_spacing_metrics']['rmse']:.4f} m")
        print(f"Overall average acceleration RMSE: {results['avg_acceleration_metrics']['rmse']:.4f} ± {results['std_acceleration_metrics']['rmse']:.4f} m/s²")
    
    return results

In [ ]:
segment_results = run_ar_segment_validation(file_path1, trace, model, train_data, segment_duration=20.0, n_samples=50)

In [ ]:
def run_dynamic_ar_segment_validation(file_path, trace, model, train_data, rho_block_info, 
                                    segment_duration=20.0, n_samples=50, d=1, step=1, tau_sec=5):

    
    
    test_data = prepare_ar_idm_data(file_path, dt_target=0.5)
    

    segments = split_data_into_segments(test_data, segment_duration=segment_duration, dt=0.5)
    

    print("Step 3: Validating all segments...")
    results = validate_dynamic_ar_model_segments(
        trace, model, train_data, segments, rho_block_info,
        n_samples=n_samples, d=d, step=step, tau_sec=tau_sec
    )
    
    if results:
        print("\n" + "="*80)
        print("FINAL DYNAMIC AR RESULTS")
        print("="*80)
        print(f"Overall average speed RMSE: {results['avg_speed_metrics']['rmse']:.4f} ± {results['std_speed_metrics']['rmse']:.4f} m/s")
        print(f"Overall average spacing RMSE: {results['avg_spacing_metrics']['rmse']:.4f} ± {results['std_spacing_metrics']['rmse']:.4f} m")
        print(f"Overall average acceleration RMSE: {results['avg_acceleration_metrics']['rmse']:.4f} ± {results['std_acceleration_metrics']['rmse']:.4f} m/s²")
        print(f"Vehicle-level average speed RMSE: {results['avg_vehicle_speed_rmse']:.4f} ± {results['std_vehicle_speed_rmse']:.4f} m/s")
        print(f"Successful segments: {results['successful_segments']}/{results['total_segments']}")
    
    return results

In [ ]:
dynamic_ar_results = run_dynamic_ar_segment_validation(
    file_path1, trace, model, train_data, rho_block_info,
    segment_duration=20.0, n_samples=50, d=1, step=1, tau_sec=5
)